### KAGGLE FEATURE ENGINEERING NOTLARI

İlk derste aralarındaki ilişkiyi daha net göstermek için bir metod olan orantısallığı gördüm. Bunu derken kastettiğim, x ve y maddesinin arasındaki fark makine için açık olmayabilir. Bunun için bizim yapmamız gerek iki kolon yerine bir kolona onların arasında oranı koymak.

İkinci derste mutual information olarak adlandırılan bir teknik gördüm. Bu teknik sklearnde bulunan bir teknik.
**sklearn.feature_selection.mutual_info_classif**

Bununla beraber aslında hedefimizdeki data ile aralarındaki bağlantıyı daha kolay görebiliriz. Bir çeşit koralasyon gibi düşünülebilir.
Araştırırken bulduğum bir kaynak var. Buradan rahatça ne olduğu anlaşılabilir.

**https://towardsdatascience.comselect-features-for-machine-learning-model-with-mutual-information-534fe387d5c8**

Mutual information olarak adlandırılan metrik 0 olduğunda hedef ve elimizdeki data arasında bir ölçülebilir bir bağlantı olmadığını gösteriyor. 2.0'ın üstündeki değerler genelde sık karşılaşılmayan değerlerdir...


![](2022-05-03-14-27-19.png)

In [ ]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores[::3] 

Üst tarafta mutual information metriği için bir skor ölçme fonksiyonu yazıyor. Altta ise bunları görselleştirmek için bir fonksiyon var...

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")


plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores)

Alttaki kodlar bu işler için iyi bir kod gösterimi olduğu için koyuyorum...

In [ ]:
features = ["YearBuilt", "MoSold", "ScreenPorch"]
sns.relplot(
    x="value", y="SalePrice", col="variable", data=df.melt(id_vars="SalePrice", value_vars=features), facet_kws=dict(sharex=False),
);

In [ ]:
feature = "GrLivArea"

sns.lmplot(
    x=feature, y="SalePrice", hue="BldgType", col="BldgType",
    data=df, scatter_kws={"edgecolor": 'w'}, col_wrap=3, height=4,
);

Bir veriyi normalize ederken kullanacağımız teknikler önemlidir. Mesela alttaki fotoğrafta, veri ile veriyi np.log1p ile normalize edilmiş hallerini görüyorsunuz...

Bu bize normal dağılıma daha yakın bir görüntü sunabilir...


![](2022-05-03-14-59-12.png)

Bazen yapacağınız işleme göre değişkenlik gösterebilecek şekilde de olsa verileri 'count' kullanarak da yapabiliriz. Mesela elimizdeki verilere bakarak olup olmamasına göre True veya False yaparak olduklarındaki değişimi bir dataframede saklayabiliriz. Bu işlerimizi kolaylaştırabilir...

Aynı şekilde işlerimizi kolaylaştırması açısından gruplayarak da datamızı şekillendirebiliriz. Aşağıda bunun için örnekler vardır...

In [ ]:
customer["AverageIncome"] = (
    customer.groupby("State")  # for each state
    ["Income"]                 # select the income
    .transform("mean")         # and compute its mean
)
#********************
customer["StateFreq"] = (
    customer.groupby("State")
    ["State"]
    .transform("count")
    / customer.State.count()
)

Tips on Creating Features
It's good to keep in mind your model's own strengths and weaknesses when creating features. Here are some guidelines:
* Linear models learn sums and differences naturally, but can't learn anything more complex.
* Ratios seem to be difficult for most models to learn. Ratio combinations often lead to some easy performance gains.
* Linear models and neural nets generally do better with normalized features. Neural nets especially need features scaled to values not too far from 0. Tree-based models (like random forests and XGBoost) can sometimes benefit from normalization, but usually much less so.
* Tree models can learn to approximate almost any combination of features, but when a combination is especially important they can still benefit from having it explicitly created, especially when data is limited.
* Counts are especially helpful for tree models, since these models don't have a natural way of aggregating information across many features at once.

### CLUSTERING WITH K-MEANS

Basitçe açıklamak gerekirse clustering(kümeleme) olarak bahsettiğimiz, benzer özelliklere sahip verilerin gruplandırılması,kümelendirilmesidir.

Aşağıdaki örnekler tek özellik ve iki özelliğe sahip clustering işlemi uygulanmış  grafiklerdir...

![](2022-05-05-14-41-59.png)

Scikit-learn'de clustering için bulunan 3 parametre var. 

**n_clusters, max_iter, n_init**

n_clusters parametresi kadar rastgele clusterla başlayan bu işlem, diğer parametrelerin optimize bir şekilde olmasıyla uzaklıkların minumum olabilecek şekilde clusterlar belirlenir. Çoğu zaman, ağırlık merkezlerinin başlangıçtaki rastgele konumu, zayıf bir kümeleme ile sonuçlanır. Bu yüzden n_init'e iyi karar verilmelidir. Ayrıca yineleme sayısı yani max_iter'in doğru noktanın bulunması için önemlidir.


![](1.gif)

In [ ]:
# Create cluster feature
kmeans = KMeans(n_clusters=6)
X["Cluster"] = kmeans.fit_predict(X)
X["Cluster"] = X["Cluster"].astype("category")


sns.relplot(
    x="Longitude", y="Latitude", hue="Cluster", data=X, height=6,
);

### Principal Component Analysis (PCA)

Kümelemenin yakınlığa dayalı olarak veri kümesinin bölümlenmesi olduğu gibi, PCA'yı da verilerdeki varyasyonun bölümlenmesi olarak düşünebilirsiniz. PCA, verilerdeki önemli ilişkileri keşfetmenize yardımcı olacak harika bir araçtır ve daha bilgilendirici özellikler oluşturmak için de kullanılabilir.

(Teknik not: PCA tipik olarak standartlaştırılmış verilere uygulanır. Standartlaştırılmış verilerle "varyasyon", "korelasyon" anlamına gelir. Standartlaştırılmamış verilerle "varyasyon", "kovaryans" anlamına gelir. Bu kurstaki tüm veriler, PCA uygulanmadan önce standartlaştırılacaktır.)

**Notice that instead of describing abalones by their 'Height' and 'Diameter', we could just as well describe them by their 'Size' and 'Shape'. This, in fact, is the whole idea of PCA: instead of describing the data with the original features, we describe it with its axes of variation. The axes of variation become the new features.**

There are two ways you could use PCA for feature engineering.

The first way is to use it as a descriptive technique. Since the components tell you about the variation, you could compute the MI scores for the components and see what kind of variation is most predictive of your target. That could give you ideas for kinds of features to create -- a product of 'Height' and 'Diameter' if 'Size' is important, say, or a ratio of 'Height' and 'Diameter' if Shape is important. You could even try clustering on one or more of the high-scoring components.

The second way is to use the components themselves as features. Because the components expose the variational structure of the data directly, they can often be more informative than the original features. Here are some use-cases:

* Dimensionality reduction: When your features are highly redundant (multicollinear, specifically), PCA will partition out the redundancy into one or more near-zero variance components, which you can then drop since they will contain little or no information.
* Anomaly detection: Unusual variation, not apparent from the original features, will often show up in the low-variance components. These components could be highly informative in an anomaly or outlier detection task.
* Noise reduction: A collection of sensor readings will often share some common background noise. PCA can sometimes collect the (informative) signal into a smaller number of features while leaving the noise alone, thus boosting the signal-to-noise ratio.
* Decorrelation: Some ML algorithms struggle with highly-correlated features. PCA transforms correlated features into uncorrelated components, which could be easier for your algorithm to work with.
PCA basically gives you direct access to the correlational structure of your data. You'll no doubt come up with applications of your own!

